# 4. Downscaling de Dados Locais e Dados do CMIP6

```python
Esse caderno tem como objetivo a obtenção da precipitação futura de dados locais 
para os pontos definidos nos GCMs do CMIP6 a partir de predição.
```

In [1]:
import os

import joblib

import pandas as pd

import numpy as np

from IPython.display import clear_output

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.cluster import KMeans

from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import (
    RandomForestRegressor, 
    ExtraTreesRegressor,
    GradientBoostingRegressor
)

from sklearn.linear_model import LinearRegression

## 4.1. Configurações

In [2]:
# Definição de se vai ocorrer ou não a geração de bases de dados
databases_generate = True

# Definição de se vai ocorrer ou não o a geração e o uso do método IDW
idw_method = True
idw_generate = False

# Tipo de base de dados local utilizada ('sum' ou 'max')
database_type = 'sum'

## 4.2. Funções

### 4.2.1. Predição a partir de Modelos de Machine Learning

In [3]:
def predicao_por_ml(df: pd.DataFrame,
                    col_de_treino: list[str],
                    var_de_predicao: str,
                    ano_X: int):

    # Definição dos modelos
    models = [

        ("ExtraTrees", ExtraTreesRegressor(
            n_estimators=200, 
            max_depth=20, 
            max_features=2,
            min_samples_split=2, 
            min_samples_leaf=1, 
            random_state=7
        )),
        
        ("RandomForest", RandomForestRegressor(
            n_estimators=300, 
            max_depth=25, 
            max_features=2, 
            min_samples_split=2, 
            min_samples_leaf=1, 
            random_state=7
        )),

        ("KNeighbors", KNeighborsRegressor(
            n_neighbors=7, 
            weights='distance', 
            algorithm='auto'
        )),
        
        ("GradientBoosting", GradientBoostingRegressor(
            n_estimators=200, 
            learning_rate=0.05, 
            max_depth=5, 
            subsample=0.8, 
            random_state=7
        )),
        
        ("LinearRegression", LinearRegression(
            fit_intercept=True, 
            positive=False
        ))
    
    ]

    # Lista de métricas por modelo
    metrics = [[0, 0] for _ in models]

    df_treino = df[df['ano'] <= ano_X].copy()
    df_teste  = df[df['ano']  > ano_X].copy()

    X_train = df_treino[col_de_treino]
    y_train = df_treino[var_de_predicao]

    X_test = df_teste[col_de_treino]
    y_test = df_teste[var_de_predicao]

    # Treinar e avaliar modelos
    for j, (nome, modelo) in enumerate(models):
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        metrics[j][0] = r2_score(y_test, y_pred)                     # R²
        metrics[j][1] = np.sqrt(mean_squared_error(y_test, y_pred))  # RMSE

    # Verificando melhor Modelo a partir de r2
    best_model = (0, '', '')

    print('Verificação de Modelos:\n')

    for i, (nome, modelo) in enumerate(models):

        r2, rmse = metrics[i][0], metrics[i][1]

        if best_model[0] < r2:
            best_model = r2, nome, modelo

        print(f'Modelo: {nome[:10]:<10} \t R²: {r2:.8f} \t RMSE: {rmse:.8f}')

    print(f'\nO melhor modelo de ML para a base de dados é: {best_model[1]}.')

    return best_model[2]

### 4.2.2. Função para Limeza de Terminal e Células

In [4]:
def clear():
    '''
    Função para limpar terminal ou célula
    '''

    # Limpando terminal
    os.system('cls')

    # Limpando célula
    clear_output(wait=True)

### 4.2.3. Interpolação a partir de Modelos de Machine Learning

In [5]:
def interpolacao_por_ml(df: pd.DataFrame,
                        col_de_treino: list[str],
                        var_de_predicao: str,
                        var_de_pontos: str,
                        n_de_teste: int):

    # Obtendo pontos únicos
    pontos_unicos = df[var_de_pontos].unique()

    # Definição dos modelos
    models = [

        ("ExtraTrees", ExtraTreesRegressor(
            n_estimators=15, 
            max_depth=20, 
            max_features=2,
            min_samples_split=2, 
            min_samples_leaf=1, 
            random_state=7
        )),
        
        ("RandomForest", RandomForestRegressor(
            n_estimators=15, 
            max_depth=25, 
            max_features=2, 
            min_samples_split=2, 
            min_samples_leaf=1, 
            random_state=7
        )),

        ("KNeighbors", KNeighborsRegressor(
            n_neighbors=7, 
            weights='distance', 
            algorithm='auto'
        )),
        
        ("GradientBoosting", GradientBoostingRegressor(
            n_estimators=200, 
            learning_rate=0.05, 
            max_depth=5, 
            subsample=0.8, 
            random_state=7
        )),
        
        ("LinearRegression", LinearRegression(
            fit_intercept=True, 
            positive=False
        ))
    
    ]

    # Defininção de lista de métricas
    metrics = []
    for i in range(len(models)):
        metrics.append([[], []])

    for i in range(n_de_teste):

        # Embaralha os pontos únicos
        np.random.shuffle(pontos_unicos)

        # Dividindo em 70% treino e 30% teste
        split_idx = int(len(pontos_unicos) * 0.8)
        pontos_treino = set(pontos_unicos[:split_idx])
        pontos_teste = set(pontos_unicos[split_idx:])

        # Criando DataFrames de treino e teste
        df_treino = df[df[var_de_pontos].isin(pontos_treino)].copy()
        df_teste = df[df[var_de_pontos].isin(pontos_teste)].copy()

        # Definindo features (X) e variável alvo (y)
        X_train = df_treino[col_de_treino]
        y_train = df_treino[var_de_predicao]

        X_test = df_teste[col_de_treino]
        y_test = df_teste[var_de_predicao]

        # Treinar e avaliar cada modelo
        for j in range(len(models)):
            models[j][1].fit(X_train, y_train)                                 # Treinamento
            y_pred = models[j][1].predict(X_test)                              # Previsão
            metrics[j][0].append(r2_score(y_test, y_pred))                     # R²
            metrics[j][1].append(np.sqrt(mean_squared_error(y_test, y_pred)))  # RMSE

    # Verificando melhor Modelo a partir de r2
    best_model = (0, '', '')

    print('Verificação de Modelos:\n')

    for i in range(len(metrics)):

        r2, rmse = np.mean(metrics[i][0]), np.mean(metrics[i][1])

        if best_model[0] < r2:
            best_model = r2, models[i][0], models[i][1]

        print(f'Modelo: {models[i][0][:3]} \t R²: {r2:.4f} \t RMSE: {rmse:.4f}')
    
    print(f'\nO melhor modelo de ML para a base de dados é: {best_model[1]}.')

    return best_model[2]

### 4.2.4. Função que Adiciona coluna IDW à Base de Dados

In [6]:
def porcentagem_em_barra(valor_atual: int, 
                         valor_total: int) -> str:
    """
    Gerador de barra de porcentagem a partir de valor atual e total.
    """
    
    porcentagem = 100 * (valor_atual / valor_total)

    completo   = '-' * (int(porcentagem))
    incompleto = '_' * (100 - int(porcentagem))

    situacao = f'[{completo}{incompleto}] {porcentagem:.2f}% ({valor_atual} de {valor_total})'

    return situacao

def vizinhos_proximos(lat: float,
                      lon: float,
                      lat_lon: list[str],
                      n_vizinhos: int = 5):
    """
    Gerador de lista de pontos vizinhos próximos de determinado ponto.

    Args:
        lat (float): Latitude do ponto principal;
        lon (float): Logitude do ponto principal;
        lat_lon (list[str]): Lista com latitudes e longitudes de pontos próximos ao ponto principal;
        n_vizinhos (int, optional): Números de pontos vizinhos ao ponto principal a se estimar.

    Returns:
        list: Lista de n pontos mais próximos ao ponto principal.
    """
    
    # Lista para armazenar tuplas (string_original, distancia)
    distancias = []

    for ponto_str in lat_lon:
        lat_p, lon_p = map(float, ponto_str.split(";"))
        dist = np.linalg.norm(np.array([lat, lon]) - np.array([lat_p, lon_p]))
        distancias.append((ponto_str, dist))
    
    # Ordena pela menor distância
    distancias_ordenadas = sorted(distancias, key=lambda x: x[1])
    
    # Pega os n vizinhos mais próximos (ignorando o primeiro se for o próprio ponto)
    vizinhos = [p[0] for p in distancias_ordenadas if p[1] != 0][:n_vizinhos]
    
    return vizinhos

def interpolacao_por_idw(df: pd.DataFrame,
                         var_de_predicao: str,
                         var_de_anos: str,
                         var_de_meses: str,
                         var_de_pontos: str,
                         pontos: str = 'all',
                         progresso: bool = True,
                         constante: int = 2,
                         n_vizinhos: int = 5) -> list:
    '''
    Interpola dados de séries temporais a partir do método IDW, e cria uma coluna para tal
    '''
    
    # Definindo nova coluna para o IDW
    df.loc[:, 'IDW'] = np.nan
    
    # Obtendo pontos únicos
    if pontos == 'all':
        pontos_unicos = df[var_de_pontos].unique()
    else:
        pontos_unicos = [pontos]

    # Varendo pontos únicos
    for i, ponto in enumerate(pontos_unicos):

        if progresso == True:
        
            # Ponto em cálculo
            print(porcentagem_em_barra(i+1, len(pontos_unicos)))

        lat, lon = map(float, ponto.split(";"))

        # Obtendo anos únicos
        anos_unicos = df[df[var_de_pontos] == ponto][var_de_anos].unique()
        
        # Varendo anos únicos dos pontos únicos
        for ano in anos_unicos:
            
            # Obtendo meses únicos
            meses_unicos = df[(df[var_de_pontos] == ponto) & 
                              (df[var_de_anos] == ano)][var_de_meses].unique()

            # Varrendo meses únicos dos anos únicos dos pontos únicos
            for mes in meses_unicos:

                # Filtrando pontos unicos que possuem mesmo mes e ano que o ponto em varredura
                pontos_unicos_filtrados = df[(df[var_de_anos] == ano) &
                                             (df[var_de_meses] == mes)][var_de_pontos].unique()
                
                # Obtendo pontos vizinhos mais próximos do ponto em varredura
                vizinhos = vizinhos_proximos(lat, lon, pontos_unicos_filtrados, n_vizinhos)

                # Definindo variáveis para calcular IDW                
                dividendo = divisor = 0

                for ponto_vizinho in vizinhos:

                    lat_vizinha, lon_vizinha = map(float, ponto_vizinho.split(";"))
                    
                    distancia = ((lat - lat_vizinha)**2 + (lon - lon_vizinha)**2)**(1/2)

                    variavel = df.loc[(df[var_de_pontos] == ponto_vizinho) & 
                                      (df[var_de_anos] == ano) & 
                                      (df[var_de_meses] == mes), var_de_predicao]
                    
                    if not variavel.empty:
                        valor = variavel.iloc[0]
                    else:
                        continue

                    dividendo += valor / (distancia**constante)

                    divisor += 1 / (distancia**constante)

                if divisor == 0:
                    idw = np.nan
                else:
                    idw = dividendo / divisor

                idx = df.loc[(df[var_de_pontos] == ponto) & 
                             (df[var_de_anos] == ano) &
                             (df[var_de_meses] == mes), 'IDW'].index[0]
                
                df.at[idx, 'IDW'] = idw
                              
        clear()

    df['IDW'] = df['IDW'].fillna(df['IDW'].mean())
                
    return df

## 4.3. CNRM-CM6-1HR

Redução de escala de dados locais e dados experimentais do CMIP6 

### 4.3.1. Coluna IDW para dados CNRM-CM6-1HR

In [7]:
# Definindo base de dados de GCM
df_cnrm_cm6_1hr = pd.read_csv(
    f"../datas/interim/1.3.2_cmip6_database_create/pr_day_CNRM-CM6-1-HR_ssp585_r1i1p1f2_gr_19940101-21001231_{database_type}.csv"
)

# Padronizando valores de longitude
df_cnrm_cm6_1hr["lon"] = df_cnrm_cm6_1hr["lon"] - 360

# Adicionando coluna de pontos
df_cnrm_cm6_1hr['pnt'] = df_cnrm_cm6_1hr["lat"].astype(str) + ";" + df_cnrm_cm6_1hr["lon"].astype(str)

# Caso não tenha a coluna IDW nas "colunas_de_interesse", calcula-se o IDW
if idw_method == True:

    if idw_generate == True:  # 115m 33.8s

        df_cnrm_cm6_1hr = interpolacao_por_idw(df_cnrm_cm6_1hr, "pr", "ano", "mes", "pnt")
        df_cnrm_cm6_1hr.to_csv(f'../datas/interim/4.3.1_cmip6_with_idw/pr_day_CNRM-CM6-1-HR_ssp585_r1i1p1f2_gr_19940101-21001231_{database_type}_idw.csv')
        
    else:
        
        df_cnrm_cm6_1hr = pd.read_csv(f'../datas/interim/4.3.1_cmip6_with_idw/pr_day_CNRM-CM6-1-HR_ssp585_r1i1p1f2_gr_19940101-21001231_{database_type}_idw.csv')

# Definindo colunas de interesse
colunas_gcm = ['lat', 'lon', 'ano', 'mes', 'pr', 'pnt']

# Restringindo base de dados às colunas de interesse
df_cnrm_cm6_1hr = df_cnrm_cm6_1hr[colunas_gcm].copy()

# Visualizando Bases de Dados do GCM
print('- Informações do GCM:')
print(df_cnrm_cm6_1hr.info())

- Informações do GCM:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64200 entries, 0 to 64199
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lat     64200 non-null  float64
 1   lon     64200 non-null  float64
 2   ano     64200 non-null  int64  
 3   mes     64200 non-null  int64  
 4   pr      64200 non-null  float64
 5   pnt     64200 non-null  object 
dtypes: float64(3), int64(2), object(1)
memory usage: 2.9+ MB
None


### 4.3.2. Criação de Bases de Dados de Redução de Escala

In [8]:
# Importando base de dados
df_aesa_to_cnrm_cm6_1hr = pd.read_csv(f'../datas/interim/3.3.3_aesa_interpolated_to_cmip6/aesa_to_cnrm_cm6_1hr_{database_type}_interpolated.csv')

# Definindo colunas de interesse
colunas_local = ['lat', 'lon', 'ano', 'mes', 'pr_local', 'pnt']

# Adicionando coluna de pontos
df_aesa_to_cnrm_cm6_1hr['pnt'] = df_aesa_to_cnrm_cm6_1hr["lat"].astype(str) + ";" + df_aesa_to_cnrm_cm6_1hr["lon"].astype(str)

# Restringindo base de dados às colunas de interesse
df_aesa_to_cnrm_cm6_1hr = df_aesa_to_cnrm_cm6_1hr[colunas_local].copy()

# Observando informações da base de dados
df_aesa_to_cnrm_cm6_1hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   lat       18000 non-null  float64
 1   lon       18000 non-null  float64
 2   ano       18000 non-null  int64  
 3   mes       18000 non-null  int64  
 4   pr_local  18000 non-null  float64
 5   pnt       18000 non-null  object 
dtypes: float64(3), int64(2), object(1)
memory usage: 843.9+ KB


In [9]:
# Fazendo o merge com base em 'pnt', 'ano' e 'mes' para base de dados única
df_aesa_to_cnrm_cm6_1hr = df_aesa_to_cnrm_cm6_1hr.merge(df_cnrm_cm6_1hr, on=['pnt', 'ano', 'mes', 'lat', 'lon'], how='outer')

# Salvando nova base de dados
df_aesa_to_cnrm_cm6_1hr.to_csv(f'../datas/interim/4.3.2_create_downscaling_database/aesa_to_cnrm_cm6_1hr_{database_type}_downscaling.csv')

# Informações da base de dados única
df_aesa_to_cnrm_cm6_1hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64200 entries, 0 to 64199
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   lat       64200 non-null  float64
 1   lon       64200 non-null  float64
 2   ano       64200 non-null  int64  
 3   mes       64200 non-null  int64  
 4   pr_local  18000 non-null  float64
 5   pnt       64200 non-null  object 
 6   pr        64200 non-null  float64
dtypes: float64(4), int64(2), object(1)
memory usage: 3.4+ MB


### 4.3.3. Configurações para Predição em Base de Dados de Redução de Escala

In [10]:
# Abrindo base de dados para configuração
df_aesa_to_cnrm_cm6_1hr = pd.read_csv(f'../datas/interim/4.3.2_create_downscaling_database/aesa_to_cnrm_cm6_1hr_{database_type}_downscaling.csv')

# Adicionando coluna de atraso de um mes da precipitação
df_aesa_to_cnrm_cm6_1hr['pr_mes_anterior'] = df_aesa_to_cnrm_cm6_1hr.sort_values(by=['pnt', 'ano', 'mes']).groupby('pnt')['pr'].shift(1)
df_aesa_to_cnrm_cm6_1hr['pr_mes_anterior'] = df_aesa_to_cnrm_cm6_1hr['pr_mes_anterior'].fillna(df_aesa_to_cnrm_cm6_1hr['pr_mes_anterior'].mean())

# Adicionando coluna de atraso de acumulado de seis meses de precipitações anteriores
df_aesa_to_cnrm_cm6_1hr['pr_acum_6m'] = df_aesa_to_cnrm_cm6_1hr.sort_values(['pnt', 'ano', 'mes']).groupby('pnt')['pr'].rolling(window=6).sum().reset_index(0, drop=True)
df_aesa_to_cnrm_cm6_1hr['pr_acum_6m'] = df_aesa_to_cnrm_cm6_1hr['pr_acum_6m'].fillna(df_aesa_to_cnrm_cm6_1hr['pr_acum_6m'].mean())

# Adicionando coluna de agrupamento de dados
df_coords = df_aesa_to_cnrm_cm6_1hr[['lat', 'lon']].drop_duplicates()
kmeans = KMeans(n_clusters=4, random_state=0).fit(df_coords)
df_coords['cluster'] = kmeans.labels_
df_aesa_to_cnrm_cm6_1hr = df_aesa_to_cnrm_cm6_1hr.merge(df_coords, on=['lat', 'lon'], how='left')

# Salvando nova base de dados gerada
df_aesa_to_cnrm_cm6_1hr.to_csv(f'../datas/interim/4.3.3_finish_downscaling_database/aesa_to_cnrm_cm6_1hr_{database_type}_downscaling_complete.csv')

# Limpando avisos
clear()

# Informações da base de dados com novas features
df_aesa_to_cnrm_cm6_1hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64200 entries, 0 to 64199
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       64200 non-null  int64  
 1   lat              64200 non-null  float64
 2   lon              64200 non-null  float64
 3   ano              64200 non-null  int64  
 4   mes              64200 non-null  int64  
 5   pr_local         18000 non-null  float64
 6   pnt              64200 non-null  object 
 7   pr               64200 non-null  float64
 8   pr_mes_anterior  64200 non-null  float64
 9   pr_acum_6m       64200 non-null  float64
 10  cluster          64200 non-null  int32  
dtypes: float64(6), int32(1), int64(3), object(1)
memory usage: 5.1+ MB


### 4.3.4. Base de Dados de Redução de Escala

In [11]:
# Abrindo base de dados para predição
df_aesa_to_cnrm_cm6_1hr.to_csv(f'../datas/interim/4.3.3_finish_downscaling_database/aesa_to_cnrm_cm6_1hr_{database_type}_downscaling_complete.csv')

# Colunas X e y
X_col, y_col = ['pr', 'pr_acum_6m', 'pr_mes_anterior', 'cluster', 'ano', 'mes', 'lat', 'lon'], "pr_local"

# Definindo ano que separará o treino e a predição
ano_X = 2017

# Supondo que sua função de predição já esteja definida:
model = predicao_por_ml(df_aesa_to_cnrm_cm6_1hr[df_aesa_to_cnrm_cm6_1hr['ano'] <= 2023], X_col, y_col, ano_X)

# Definindo features (X) e variável alvo (y)
X = df_aesa_to_cnrm_cm6_1hr[df_aesa_to_cnrm_cm6_1hr['ano'] <= 2023][X_col].copy()
y = df_aesa_to_cnrm_cm6_1hr[df_aesa_to_cnrm_cm6_1hr['ano'] <= 2023][y_col].copy()

# Treinamento
model.fit(X, y)

Verificação de Modelos:

Modelo: ExtraTrees 	 R²: 0.59100460 	 RMSE: 56.28908754
Modelo: RandomFore 	 R²: 0.59114112 	 RMSE: 56.27969184
Modelo: KNeighbors 	 R²: 0.57439228 	 RMSE: 57.42086658
Modelo: GradientBo 	 R²: 0.59980941 	 RMSE: 55.67989769
Modelo: LinearRegr 	 R²: 0.37759311 	 RMSE: 69.43879713

O melhor modelo de ML para a base de dados é: GradientBoosting.


GradientBoostingRegressor(learning_rate=0.05, max_depth=5, n_estimators=200,
                          random_state=7, subsample=0.8)

In [12]:
df_aesa_to_cnrm_cm6_1hr[df_aesa_to_cnrm_cm6_1hr['ano'] <= 2023].info()

<class 'pandas.core.frame.DataFrame'>
Index: 18000 entries, 0 to 63275
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       18000 non-null  int64  
 1   lat              18000 non-null  float64
 2   lon              18000 non-null  float64
 3   ano              18000 non-null  int64  
 4   mes              18000 non-null  int64  
 5   pr_local         18000 non-null  float64
 6   pnt              18000 non-null  object 
 7   pr               18000 non-null  float64
 8   pr_mes_anterior  18000 non-null  float64
 9   pr_acum_6m       18000 non-null  float64
 10  cluster          18000 non-null  int32  
dtypes: float64(6), int32(1), int64(3), object(1)
memory usage: 1.6+ MB


In [13]:
# Previsão dos valores de 'pr' com o modelo treinado
pr_local = model.predict(df_aesa_to_cnrm_cm6_1hr[df_aesa_to_cnrm_cm6_1hr['ano'] > 2023][X_col])

# Adicionando a nova coluna 'pr' ao DataFrame
df_aesa_to_cnrm_cm6_1hr.loc[df_aesa_to_cnrm_cm6_1hr[y_col].isnull(), y_col] = pr_local

df_aesa_to_cnrm_cm6_1hr.to_csv(f'../datas/processed/4.3.4_downscaling_database/aesa_to_cnrm_cm6_1hr_{database_type}_downscaling_database.csv')

# Informações da base de dados predita
df_aesa_to_cnrm_cm6_1hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64200 entries, 0 to 64199
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       64200 non-null  int64  
 1   lat              64200 non-null  float64
 2   lon              64200 non-null  float64
 3   ano              64200 non-null  int64  
 4   mes              64200 non-null  int64  
 5   pr_local         64200 non-null  float64
 6   pnt              64200 non-null  object 
 7   pr               64200 non-null  float64
 8   pr_mes_anterior  64200 non-null  float64
 9   pr_acum_6m       64200 non-null  float64
 10  cluster          64200 non-null  int32  
dtypes: float64(6), int32(1), int64(3), object(1)
memory usage: 5.1+ MB


### 4.3.5. Geração de Modelo Preditivo Final de Interpolação de Base de Dados de Redução de Escala

In [14]:
df_aesa_to_cnrm_cm6_1hr.to_csv(f'../datas/processed/4.3.4_downscaling_database/aesa_to_cnrm_cm6_1hr_{database_type}_downscaling_database.csv')

# Colunas X e y
X_col, y_col = ['ano', 'mes', 'lat', 'lon'], "pr_local"

# Escolhendo melhor modelo preditivo
model = interpolacao_por_ml(df_aesa_to_cnrm_cm6_1hr, X_col, y_col, "pnt", 1)

# Definindo features (X) e variável alvo (y)
X = df_aesa_to_cnrm_cm6_1hr[X_col].copy()
y = df_aesa_to_cnrm_cm6_1hr[y_col].copy()

# Treinamento
model.fit(X, y)

# Salvando o modelo
joblib.dump(model, f'../models/aesa_to_cnrm_cm6_1hr_{database_type}_downscaling_database.joblib', compress=3)

Verificação de Modelos:

Modelo: Ext 	 R²: 0.9324 	 RMSE: 17.4166
Modelo: Ran 	 R²: 0.9234 	 RMSE: 18.5338
Modelo: KNe 	 R²: 0.9322 	 RMSE: 17.4450
Modelo: Gra 	 R²: 0.8810 	 RMSE: 23.1106
Modelo: Lin 	 R²: 0.3478 	 RMSE: 54.0957

O melhor modelo de ML para a base de dados é: ExtraTrees.


['../models/aesa_to_cnrm_cm6_1hr_sum_downscaling_database.joblib']